In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
#import lightgbm as lgb

# Data imputation
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.model_selection import GridSearchCV, KFold

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

sns.set_style("whitegrid")

import warnings
warnings.filterwarnings('ignore')

import dengue_utils as dutils
import dsutils as du

import xgboost as xgb

In [2]:
# Load dataset

features_train = pd.read_csv('../data/dengue_features_train.csv')
labels_train   = pd.read_csv('../data/dengue_labels_train.csv')

features_test = pd.read_csv('../data/dengue_features_test.csv')

dengue_dataset = pd.concat([features_train, features_test], axis = 0)

# Join train labes and features
dengue_dataset_labeled = dengue_dataset.merge(labels_train, 
                                           left_on = ['city', 'year', 'weekofyear'], 
                                           right_on = ['city', 'year', 'weekofyear'],
                                           how = 'outer')

In [3]:
dengue_dataset_labeled['week_start_date'] = pd.to_datetime(dengue_dataset_labeled['week_start_date'])

In [4]:
dengue_dataset_labeled.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4.0
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5.0
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,4.0
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,3.0
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,6.0


In [6]:
series = dengue_dataset_labeled.loc[:,['week_start_date', 'total_cases']]
series = series.set_index('week_start_date')

In [7]:
series.head()

,total_cases
week_start_date,
1990-04-30,4.0
1990-05-07,5.0
1990-05-14,4.0
1990-05-21,3.0
1990-05-28,6.0


In [11]:
series.dropna().describe()

,total_cases
count,1456.000000
mean,24.675137
std,43.596000
min,0.000000
25%,5.000000
50%,12.000000
75%,28.000000
max,461.000000


In [8]:
### Prueba con ARIMA

In [20]:
from statsmodels.tsa.arima_model import ARIMA

# fit model
model = ARIMA(series.total_cases, order=(2,1,0))
model_fit = model.fit(disp=-1)
print(model_fit.summary())

LinAlgError: Array must not contain infs or NaNs

In [12]:
# plot residual errors
residuals = DataFrame(model_fit.resid)
residuals.plot()
pyplot.show()
residuals.plot(kind='kde')
pyplot.show()
print(residuals.describe())

LinAlgError: Array must not contain infs or NaNs